# CSE314 Final Project

In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
from glob import glob
import re
import json
import tqdm
import ast
import dash
import os
import sys


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ModuleNotFoundError: No module named 'scipy'

In [2]:
sys.path.append('/home/themaster/workspace/cse314final/MovieClean')
from MovieClean import norm

module initialized


In [3]:
path = "/media/work_data/DS_final_project"
file_names = !ls $path
file_names

['ls: /media/work_data/DS_final_project: No such file or directory']

In [4]:
save_path = './Data/'

# Clean Credits

In [6]:
credits = pd.read_csv(path+"/"+file_names[1])
credits

IndexError: list index out of range

In [6]:
all_cast = norm.json2df(credits, 'cast', 'id')
all_cast

,cast_id,character,credit_id,gender,id,name,order,profile_path
0,14.0,Woody (voice),52fe4284c3a36847f8024f95,2.0,862.0,Tom Hanks,0.0,/pQFoyx7rp09CJTAb932F2g8Nlho.jpg
1,15.0,Buzz Lightyear (voice),52fe4284c3a36847f8024f99,2.0,862.0,Tim Allen,1.0,/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg
2,16.0,Mr. Potato Head (voice),52fe4284c3a36847f8024f9d,2.0,862.0,Don Rickles,2.0,/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg
3,17.0,Slinky Dog (voice),52fe4284c3a36847f8024fa1,2.0,862.0,Jim Varney,3.0,/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg
4,18.0,Rex (voice),52fe4284c3a36847f8024fa5,2.0,862.0,Wallace Shawn,4.0,/oGE6JqPP2xH4tNORKNqxbNPYi7u.jpg
...,...,...,...,...,...,...,...,...
0,2.0,,52fe4ea59251416c7515d7d5,2.0,227506.0,Iwan Mosschuchin,0.0,None
1,3.0,,52fe4ea59251416c7515d7d9,1.0,227506.0,Nathalie Lissenko,1.0,None
2,4.0,,52fe4ea59251416c7515d7dd,2.0,227506.0,Pavel Pavlov,2.0,None
3,5.0,,52fe4ea59251416c7515d7e1,0.0,227506.0,Aleksandr Chabrov,3.0,None


In [7]:
actors = norm.split_table(all_cast, 'name', ['character',  'gender', 'id'])
actors.reset_index(inplace=True)
actors['name'] = actors['name'].str.upper()
actors['id'] = actors['id'].astype(int)
actors.set_index(['id', 'name'], inplace=True)
actors

character  gender
id     name                                              
862    TOM HANKS                    Woody (voice)     2.0
       TIM ALLEN           Buzz Lightyear (voice)     2.0
       DON RICKLES        Mr. Potato Head (voice)     2.0
       JIM VARNEY              Slinky Dog (voice)     2.0
       WALLACE SHAWN                  Rex (voice)     2.0
...                                           ...     ...
227506 IWAN MOSSCHUCHIN                               2.0
       NATHALIE LISSENKO                              1.0
       PAVEL PAVLOV                                   2.0
       ALEKSANDR CHABROV                              0.0
       VERA ORLOVA                                    1.0

[562474 rows x 2 columns]

In [8]:
#Checkpoint
actors.to_csv(save_path+'actors.csv', encoding='utf-8')

# Clean Keywords

In [9]:
keywords = pd.read_csv(path+"/"+file_names[2])
keywords

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."
...,...,...
46414,439050,"[{'id': 10703, 'name': 'tragic love'}]"
46415,111109,"[{'id': 2679, 'name': 'artist'}, {'id': 14531,..."
46416,67758,[]
46417,227506,[]


In [10]:
keywords = norm.json2series(keywords, 'keywords', 'name', 'id').rename('keyword')
keywords = keywords.str.upper()
keywords

/home/themaster/workspace/cse314final/MovieClean/MovieClean/norm.py:86: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  .apply(pd.Series)


id
862                JEALOUSY
8844             BOARD GAME
15602               FISHING
31357        BASED ON NOVEL
11862                  BABY
                ...        
23160         MUSICAL NOTES
23160                ERRAND
23160        DEATH OF UNCLE
23160               STRANGE
23160    RUSSIAN OCCUPATION
Name: keyword, Length: 158680, dtype: object

In [12]:
#Checkpoint
keywords.to_csv('./Data/keywords.csv', encoding='utf-8')

# Clean Movies Metadata

In [13]:
movies = pd.read_csv(path+"/"+file_names[5])
movies.head()

/tmp/ipykernel_9163/663887663.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv(path+"/"+file_names[5])


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [14]:
collections = (
    norm.json2series(movies[['belongs_to_collection', 'id']]
                  .dropna(), 'belongs_to_collection', 'name', 'id')
    .rename('collection')
)
collections = collections.str.replace(r'Collection', '').str.upper()
collections

id
862                 TOY STORY 
15602          GRUMPY OLD MEN 
11862     FATHER OF THE BRIDE 
710                JAMES BOND 
21032                   BALTO 
                  ...         
24568            THE CARRY ON 
19307            THE CARRY ON 
21251            THE CARRY ON 
460135    DC SUPER HERO GIRLS 
182030              RED LOTUS 
Name: collection, Length: 4494, dtype: object

In [20]:
collections = norm.drop_duplicate_index(collections)
assert collections.index.is_unique 

In [21]:
collections = norm.id2int(collections)

In [22]:
genres = norm.json2series(movies[['genres', 'id']].dropna(), 'genres', 'name', 'id').rename('genre')
genres = norm.id2int(genres)
genres = genres.squeeze()
genres = genres.str.upper()
genres

/home/themaster/workspace/cse314final/MovieClean/MovieClean/norm.py:86: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  .apply(pd.Series)


id
862       ANIMATION
8844      ADVENTURE
15602       ROMANCE
31357        COMEDY
11862        COMEDY
            ...    
220447        DRAMA
90063     ANIMATION
11052        FAMILY
65592        COMEDY
325712     THRILLER
Name: genre, Length: 91094, dtype: object

In [23]:
languages = norm.json2series(movies[['id', 'spoken_languages']].dropna(), 'spoken_languages', 'name', 'id').rename('language')
languages = norm.id2int(languages)
languages = languages.squeeze()
languages = languages.str.upper()
languages

/home/themaster/workspace/cse314final/MovieClean/MovieClean/norm.py:86: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  .apply(pd.Series)


id
862       ENGLISH
8844      ENGLISH
15602     ENGLISH
31357     ENGLISH
11862     ENGLISH
           ...   
8985      ESPAÑOL
8985      SVENSKA
8985       TÜRKÇE
8985     ΕΛΛΗΝΙΚΆ
8985      ENGLISH
Name: language, Length: 53300, dtype: object

In [24]:
companies = norm.json2series(movies[['id','production_companies']].dropna(), 'production_companies', 'name', 'id').rename('company')
companies = norm.id2int(companies)
companies = companies.squeeze()
companies = companies.str.upper()
companies

/home/themaster/workspace/cse314final/MovieClean/MovieClean/norm.py:86: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  .apply(pd.Series)
/home/themaster/workspace/cse314final/MovieClean/MovieClean/norm.py:86: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  .apply(pd.Series)
/home/themaster/workspace/cse314final/MovieClean/MovieClean/norm.py:86: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  .apply(pd.Series)
/home/themaster/workspace/cse314final/MovieClean/MovieClean/norm.py:86: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  .apply

id
862                      PIXAR ANIMATION STUDIOS
8844                            TRISTAR PICTURES
15602                               WARNER BROS.
31357     TWENTIETH CENTURY FOX FILM CORPORATION
11862                      SANDOLLAR PRODUCTIONS
                           ...                  
8985                                       SHOTS
345775                             CPH FILM FUND
16                   LANTIA CINEMA & AUDIOVISIVI
8985                              TITANIC S.R.O.
345775                  ASSOCIATION BEAUMARCHAIS
Name: company, Length: 70545, dtype: object

In [25]:
countries = norm.json2series(movies[['id','production_countries']].dropna(), 'production_countries', 'name', 'id').rename('country')
countries = norm.id2int(countries)
countries = countries.squeeze()
countries = countries.str.upper()
countries

/home/themaster/workspace/cse314final/MovieClean/MovieClean/norm.py:86: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  .apply(pd.Series)
/home/themaster/workspace/cse314final/MovieClean/MovieClean/norm.py:86: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  .apply(pd.Series)
/home/themaster/workspace/cse314final/MovieClean/MovieClean/norm.py:86: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  .apply(pd.Series)
/home/themaster/workspace/cse314final/MovieClean/MovieClean/norm.py:86: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  .apply

id
862      UNITED STATES OF AMERICA
8844     UNITED STATES OF AMERICA
15602    UNITED STATES OF AMERICA
31357    UNITED STATES OF AMERICA
11862    UNITED STATES OF AMERICA
                   ...           
8985                     SLOVAKIA
8985                     SLOVENIA
8985                        SPAIN
8985                       SWEDEN
8985               UNITED KINGDOM
Name: country, Length: 49423, dtype: object

In [34]:
clean_movie = movies.drop(columns=['belongs_to_collection', 'genres', 'homepage', 'spoken_languages', 'tagline', 'production_companies', 'production_countries', 'poster_path']).set_index('id')
clean_movie = norm.id2int(clean_movie)
clean_movie['title'] = clean_movie['title'].str.upper()
clean_movie = clean_movie.join(collections)
clean_movie

,adult,budget,imdb_id,original_language,original_title,overview,popularity,release_date,revenue,runtime,status,title,video,vote_average,vote_count,collection
id,,,,,,,,,,,,,,,,
2,False,0,tt0094675,fi,Ariel,Taisto Kasurinen is a Finnish coal miner whose...,3.860491,1988-10-21,0.0,69.0,Released,ARIEL,False,7.1,44.0,NaN
3,False,0,tt0092149,fi,Varjoja paratiisissa,"An episode in the life of Nikander, a garbage ...",2.29211,1986-10-16,0.0,76.0,Released,SHADOWS IN PARADISE,False,7.1,35.0,NaN
5,False,4000000,tt0113101,en,Four Rooms,It's Ted the Bellhop's first night on the job....,9.026586,1995-12-09,4300000.0,98.0,Released,FOUR ROOMS,False,6.5,539.0,NaN
6,False,0,tt0107286,en,Judgment Night,"While racing to a boxing match, Frank, Mike, J...",5.538671,1993-10-15,12136938.0,110.0,Released,JUDGMENT NIGHT,False,6.4,79.0,NaN
11,False,11000000,tt0076759,en,Star Wars,Princess Leia is captured and held hostage by ...,42.149697,1977-05-25,775398007.0,121.0,Released,STAR WARS,False,8.1,6778.0,STAR WARS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
464819,False,0,tt0365473,en,The Making of 'The Terminator': A Retrospective,An interview session with Arnold Schwarzenegge...,1e-06,1992-01-01,0.0,18.0,Released,THE MAKING OF 'THE TERMINATOR': A RETROSPECTIVE,False,0.0,0.0,NaN
465044,False,0,tt5943940,en,Abduction,A horror comedy spoofing conspiracy theory mov...,0.281008,2017-06-28,0.0,90.0,Released,ABDUCTION,False,0.0,0.0,NaN
467731,False,0,tt0507700,en,Tragedy in a Temporary Town,Fifteen-year-old girl Dotty Fisher is assaulte...,0.001189,1956-02-19,0.0,60.0,Released,TRAGEDY IN A TEMPORARY TOWN,False,0.0,0.0,NaN


In [35]:
clean_movie = norm.drop_duplicate_index(clean_movie)
clean_movie

,adult,budget,imdb_id,original_language,original_title,overview,popularity,release_date,revenue,runtime,status,title,video,vote_average,vote_count,collection
id,,,,,,,,,,,,,,,,
2,False,0,tt0094675,fi,Ariel,Taisto Kasurinen is a Finnish coal miner whose...,3.860491,1988-10-21,0.0,69.0,Released,ARIEL,False,7.1,44.0,NaN
3,False,0,tt0092149,fi,Varjoja paratiisissa,"An episode in the life of Nikander, a garbage ...",2.29211,1986-10-16,0.0,76.0,Released,SHADOWS IN PARADISE,False,7.1,35.0,NaN
5,False,4000000,tt0113101,en,Four Rooms,It's Ted the Bellhop's first night on the job....,9.026586,1995-12-09,4300000.0,98.0,Released,FOUR ROOMS,False,6.5,539.0,NaN
6,False,0,tt0107286,en,Judgment Night,"While racing to a boxing match, Frank, Mike, J...",5.538671,1993-10-15,12136938.0,110.0,Released,JUDGMENT NIGHT,False,6.4,79.0,NaN
11,False,11000000,tt0076759,en,Star Wars,Princess Leia is captured and held hostage by ...,42.149697,1977-05-25,775398007.0,121.0,Released,STAR WARS,False,8.1,6778.0,STAR WARS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
464819,False,0,tt0365473,en,The Making of 'The Terminator': A Retrospective,An interview session with Arnold Schwarzenegge...,1e-06,1992-01-01,0.0,18.0,Released,THE MAKING OF 'THE TERMINATOR': A RETROSPECTIVE,False,0.0,0.0,NaN
465044,False,0,tt5943940,en,Abduction,A horror comedy spoofing conspiracy theory mov...,0.281008,2017-06-28,0.0,90.0,Released,ABDUCTION,False,0.0,0.0,NaN
467731,False,0,tt0507700,en,Tragedy in a Temporary Town,Fifteen-year-old girl Dotty Fisher is assaulte...,0.001189,1956-02-19,0.0,60.0,Released,TRAGEDY IN A TEMPORARY TOWN,False,0.0,0.0,NaN


# Encode Status 

In [36]:
status_map = norm.cat_encode(clean_movie, 'status')
status_map

/home/themaster/workspace/cse314final/MovieClean/MovieClean/norm.py:138: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, col_name] = data[col_name].apply(lambda x: mapping[x])
/home/themaster/workspace/cse314final/MovieClean/MovieClean/norm.py:138: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data.loc[:, col_name] = data[col_name].apply(lambda x: mapping[x])


0           Canceled
1      In Production
2            Planned
3    Post Production
4           Released
5            Rumored
Name: status_map, dtype: object

# Nullify 0's


In [44]:
clean_movie.loc[:, 'budget'] = clean_movie['budget'].astype(float)
clean_movie.loc[clean_movie['budget']==0, 'budget'] = np.NaN
clean_movie.loc[clean_movie['revenue']==0, 'revenue'] = np.NaN
clean_movie.loc[clean_movie['vote_count']==0, 'vote_average'] = np.NaN
clean_movie

/tmp/ipykernel_9163/2577780981.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_movie.loc[:, 'budget'] = clean_movie['budget'].astype(float)


,adult,budget,imdb_id,original_language,original_title,overview,popularity,release_date,revenue,runtime,status,title,video,vote_average,vote_count,collection
id,,,,,,,,,,,,,,,,
2,False,NaN,tt0094675,fi,Ariel,Taisto Kasurinen is a Finnish coal miner whose...,3.860491,1988-10-21,NaN,69.0,4,ARIEL,False,7.1,44.0,NaN
3,False,NaN,tt0092149,fi,Varjoja paratiisissa,"An episode in the life of Nikander, a garbage ...",2.29211,1986-10-16,NaN,76.0,4,SHADOWS IN PARADISE,False,7.1,35.0,NaN
5,False,4000000.0,tt0113101,en,Four Rooms,It's Ted the Bellhop's first night on the job....,9.026586,1995-12-09,4300000.0,98.0,4,FOUR ROOMS,False,6.5,539.0,NaN
6,False,NaN,tt0107286,en,Judgment Night,"While racing to a boxing match, Frank, Mike, J...",5.538671,1993-10-15,12136938.0,110.0,4,JUDGMENT NIGHT,False,6.4,79.0,NaN
11,False,11000000.0,tt0076759,en,Star Wars,Princess Leia is captured and held hostage by ...,42.149697,1977-05-25,775398007.0,121.0,4,STAR WARS,False,8.1,6778.0,STAR WARS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
464819,False,NaN,tt0365473,en,The Making of 'The Terminator': A Retrospective,An interview session with Arnold Schwarzenegge...,1e-06,1992-01-01,NaN,18.0,4,THE MAKING OF 'THE TERMINATOR': A RETROSPECTIVE,False,NaN,0.0,NaN
465044,False,NaN,tt5943940,en,Abduction,A horror comedy spoofing conspiracy theory mov...,0.281008,2017-06-28,NaN,90.0,4,ABDUCTION,False,NaN,0.0,NaN
467731,False,NaN,tt0507700,en,Tragedy in a Temporary Town,Fifteen-year-old girl Dotty Fisher is assaulte...,0.001189,1956-02-19,NaN,60.0,4,TRAGEDY IN A TEMPORARY TOWN,False,NaN,0.0,NaN


In [45]:
#Checkpoint
clean_movie.to_csv(save_path+'clean_movie.csv', encoding='utf-8')
countries.to_csv(save_path+'countries.csv', encoding='utf-8')
companies.to_csv(save_path+'companies.csv', encoding='utf-8')
languages.to_csv(save_path+'languages.csv', encoding='utf-8')
genres.to_csv(save_path+'genres.csv', encoding='utf-8')
status_map.to_csv(save_path+'status_map.csv', encoding='utf-8')

# clean ratings

In [46]:
ratings = pd.read_csv(path+"/"+file_names[6])
ratings.head()

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


In [47]:
ratings = ratings.groupby('movieId')['rating'].mean()

In [48]:
ratings.index.name='id'

In [49]:
ratings.index

Int64Index([     1,      2,      3,      4,      5,      6,      7,      8,
                 9,     10,
            ...
            176253, 176255, 176257, 176259, 176263, 176267, 176269, 176271,
            176273, 176275],
           dtype='int64', name='id', length=45115)

In [50]:
ratings.to_csv(save_path+'ratings.csv', encoding='utf-8')